In [2]:
import Pkg
Pkg.add("HiGHS")

    Updating registry at `~/Documents/master/MAOA/julia-1.8.2/.julia_commun_a_tous/registries/General.toml`
   Resolving package versions...
   Installed HiGHS_jll ─ v1.4.0+0
   Installed HiGHS ───── v1.3.0
    Updating `~/Documents/master/MAOA/julia-1.8.2/.julia_commun_a_tous/environments/v1.8/Project.toml`
  [87dc4568] + HiGHS v1.3.0
    Updating `~/Documents/master/MAOA/julia-1.8.2/.julia_commun_a_tous/environments/v1.8/Manifest.toml`
  [87dc4568] + HiGHS v1.3.0
  [8fd58aa0] + HiGHS_jll v1.4.0+0
Precompiling project...
  ✓ HiGHS_jll
  ✓ HiGHS
  2 dependencies successfully precompiled in 6 seconds. 203 already precompiled. 1 skipped during auto due to previous errors.


In [3]:
import HiGHS

In [4]:
using JuMP
using CPLEX
using CPUTime
using Graphs


In [5]:
struct Clients
    nb_clients
    X
    Y
    h
    L
    L0
    d
end

In [190]:
function Read_PRP_instance(filename)
    hashmap = Dict()
    open(filename) do f
        
        demands = false
        listing_active = false
        current_value = 0
        
        nbc = 0
        X = Array{Float64}(undef, 0)
        Y = Array{Float64}(undef, 0)
        h = Array{Float64}(undef, 0)
        L = Array{Float64}(undef, 0)
        L0 = Array{Float64}(undef, 0)
        d = Array{Array{Float64}}(undef, 0)
        
        for (i, line) in enumerate(eachline(f))
            x = split(line, " ")
            deleteat!(x, findall(e->e=="", x))
            current_item = x[1]
            
            if(current_item == "0" || listing_active) # Liste de clients / demandes
                listing_active = true
                if(current_item == "d")
                    push!(d, zeros(hashmap["l"]+1))
                    demands = true
                elseif(demands) # Enregistrement des demandes
                    push!(d, append!([0],parse.(Float64, x)[2:end]))
                else # enregistrement des clients
                    nbc += 1
                    push!(X, parse(Float64, x[2]))
                    push!(Y, parse(Float64, x[3]))
                    push!(h, parse(Float64, x[6]))
                    push!(L, parse(Float64, x[8]))
                    push!(L0, parse(Float64, x[10]))
                end
            else # variables générales du problème
                try
                    current_value = parse(Int64, x[2])
                catch err
                    current_value = 10000000000
                end
                hashmap[current_item] = current_value
            end
        end
        C = Clients(nbc, X, Y, h, L, L0, d)
        hashmap["Clients"] = C
    end 
    return hashmap
end

Read_PRP_instance (generic function with 1 method)

In [191]:
h = Read_PRP_instance("./PRP_instances/A_014_#ABS1_15_1.prp")

Dict{Any, Any} with 9 entries:
  "f"       => 3000
  "Q"       => 322
  "C"       => 10000000000
  "k"       => 2085
  "Type"    => 1
  "u"       => 30
  "l"       => 6
  "Clients" => Clients(15, [143.0, 89.0, 76.0, 285.0, 401.0, 16.0, 267.0, 249.0…
  "n"       => 14

In [9]:
function dist(C, i, j, t, mc)
    if(t == 1)
        return ((C.X[i] - C.X[j])^2 + (C.Y[i] - C.Y[j])^2)^(0.5) + 0.5
    else
        return ((C.X[i] - C.X[j])^2 + (C.Y[i] - C.Y[j])^2)^(0.5) * mc
    end
end

dist (generic function with 1 method)

In [10]:
function calcul_dist(p)
    if(p["Type"] == 1)
        mc = 0
    else
        mc = p["mc"]
    end
    c = Array{Float64}(undef, (p["n"]+1, p["n"]+1))
    for i in 1:p["n"]+1
        for j in 1:p["n"]+1            
            c[i, j] = dist(p["Clients"], i, j, p["Type"], mc)
        end
    end
    return c
end

calcul_dist (generic function with 1 method)

In [11]:
sum(h["Clients"].h[i] for i in 1:15)

106.0

In [46]:
sum(sum(h["Clients"].d[jj][tt] for jj in 1:h["Clients"].nb_clients) for tt in 1:h["l"])

1380.0

In [13]:
for i in 1:6
    print(i)
end

123456

In [14]:
h["Clients"].nb_clients

15

In [192]:
h["Clients"].d

15-element Vector{Array{Float64}}:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
 [0.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0]
 [0.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0]
 [0.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0]
 [0.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0]
 [0.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0]
 [0.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0]
 [0.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0]
 [0.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0]
 [0.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0]
 [0.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0]
 [0.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0]
 [0.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0]
 [0.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0]

In [16]:
a = [1, 2]
b = [1,2,3]
c = b[b .∉ Ref(a)]
print(c)

[3]

In [17]:
function partiesliste(seq, min_size, max_size)
    p = [] #Array{Float64}(undef, 2^size(seq)[1]-1 - 2^min_size)
    i = 1
    imax = 2 ^ size(seq)[1]
    while(i <= imax)
        s = []
        j, jmax = 0, size(seq)[1]-1
        while(j <= jmax)
            if((i>>j)&1 == 1)
                append!(s, seq[j+1])
            end
            j += 1
        end
        if(size(s)[1] >= min_size && size(s)[1] <= max_size)
            append!(p, [s])
        end
        i += 1
    end
    return p
end

partiesliste (generic function with 1 method)

In [202]:
function Resolve_PRP(p)
    # LP = Model(HiGHS.Optimizer)
    LP = Model(CPLEX.Optimizer)
    
    c = calcul_dist(p)
    
    n = p["Clients"].nb_clients      # Nombre de clients (usine inclue)
    l = p["l"]                       # Nombre de périodes
    
    @variable(LP, y[1:l], Bin)  # Production démarrée
    @variable(LP, prod[1:l], Int)  # Quantité produite
    @variable(LP, inv[1:n, 1:l+1], Int)  # Inventaire des clients
    @variable(LP, x[1:n, 1:n, 1:l], Bin)  # Camion entre i et j au temps t
    @variable(LP, q[1:n, 1:l], Int) # qt délivré au client i à la période t
    @variable(LP, z[1:n, 1:l], Int) # 1 client i visité au temps t, 0 sinon (Attention z[0, t] nb de véhicules 
                                    # qui quittent l'usine au temps t)
    @variable(LP, w[1:n, 1:l], Int) # chargement véhicule avant de faire une livraison au client i au temps t 

    @objective(LP, Min, sum(p["u"]*prod[t] + p["f"]*y[t] + 
            sum(p["Clients"].h[i]*inv[i, t] for i in 1:n) + 
            sum(sum(c[i, j] * x[i, j, t] for i in 1:n if i ≠ j) for j in 1:n) for t in 1:l))
    
    indices_array = [i for i in 1:n-1]
    allS = partiesliste(indices_array, 1, n-2)
    
    for i in 1:n
        @constraint(LP, inv[i, 1] == p["Clients"].L0[i]) # initialise les stocks
        @constraint(LP, 0 <= inv[i, l+1]) # (13.2.2) OK
    end
    
    for t in 1:l # Pour toutes les périodes (l=nb de prériodes)
        
        ############################
        # Contraintes d'intervalle #
        ############################
        for i in 1:n
            @constraint(LP, 0 <= inv[i, t]) # (13.2.1) OK
            @constraint(LP, 0 <= q[i, t]) # (13.3) OK 
            # @constraint(LP, 0 <= x[i, j, t] <= 1) # (14.2) OK 
            if(i ≠ 1)
                @constraint(LP, 0 <= z[i, t] <= 1)
            end
        end
        
        ######################
        # Lot-sizing problem #
        ######################
        if(t ≠ 1) 
            @constraint(LP, inv[1, t] + prod[t] == sum(q[i, t] for i in 2:n) + inv[1, t+1]) # (2) OK OK
        end
        
        for i in 1:n # Parcours de tous les clients
            xijt_sum = sum(x[i, jj, t] for jj in 1:n)
            sum_d_current_i = sum(p["Clients"].d[i][tt] for tt in 1:l)
            Mit = min(p["Clients"].L[i], p["Q"], sum_d_current_i)
            if(i ≠ 1) # Nc
                tt = t+1
                @constraint(LP, inv[i, t] + q[i, t] == p["Clients"].d[i][t] + inv[i, t+1]) # (3) OK OK
                if(t ≠ 1)
                    @constraint(LP, inv[i, t-1] + q[i, t] <= p["Clients"].L[i]) # (6) OK OK
                end
                @constraint(LP, q[i, t] <= Mit * z[i, t]) # (7) OK
                @constraint(LP, xijt_sum == z[i, t]) # (8) OK
                @constraint(LP, 0 <= w[i, t]) # (12.1) OK
                @constraint(LP, w[i, t] <= p["Q"] * z[i, t]) # (12.2) OK
            end
            for j in 1:n
                if(i ≠ j)
                    @constraint(LP, w[i, t] - w[j, t] <= q[i, t] - Mit * (1 - x[i, j, t])) # (11) OK
                end
            end
            @constraint(LP, xijt_sum + sum(x[jj, i, t] for jj in 1:n) == 2 * z[i, t]) # (9) OK
        end
        sum_d_current_t = sum(sum(p["Clients"].d[ii][tt] for ii in 2:n) for tt in t:l) 
        Mt = min(p["C"], sum_d_current_t) # OK
        @constraint(LP, prod[t] <= Mt * y[t]) # (4) OK OK
        @constraint(LP, 0 <= prod[t]) # (13.1) OK 
        @constraint(LP, inv[1, t] <= p["Clients"].L[1]) # (5) OK OK
        """for ii in allS
            jj = indices_array[indices_array .∉ Ref(ii)]
            # (17) OK :
            @constraint(LP, sum(sum(x[i, j, t] for j in jj) for i in ii) >= sum(q[i, t] for i in ii) / p["Q"])
            if(size(ii)[1] >= 2 && size(ii)[1] <= n-3)
                @constraint(LP, sum(sum(x[i, j, t] for j in ii) for i in ii) <= 
                    size(ii)[1] - sum(q[i, t] for i in ii) / p["Q"]) # (18) OK
                @constraint(LP, p["Q"] * sum(sum(x[i, j, t] for j in ii) for i in ii) <= 
                    sum(p["Q"] * z[i, t] - q[i, t] for i in ii)) # (19) OK
            end
        end"""
    end
        
    
    # print(LP)
    # println()
    optimize!(LP)
   
    println(solution_summary(LP, verbose=true))
    
    return LP
end

Resolve_PRP (generic function with 1 method)

In [203]:
LP = Resolve_PRP(Read_PRP_instance("./PRP_instances/A_014_#ABS1_15_1.prp"))

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Tried aggregator 2 times.
MIP Presolve eliminated 573 rows and 113 columns.
MIP Presolve modified 1409 coefficients.
Aggregator did 53 substitutions.
Reduced MIP has 1531 rows, 1655 columns, and 7932 nonzeros.
Reduced MIP has 1439 binaries, 216 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (9.59 ticks)
* Solver : CPLEX

* Status
  Termination status : INFEASIBLE
  Primal status      : NO_SOLUTION
  Dual status        : NO_SOLUTION
  Result count       : 0
  Has duals          : false
  Message from the solver:
  "integer infeasible"

* Candidate solution
  Objective bound      : 1.00000e+75

* Work counters
  Solve time (sec)   : 4.56462e-02
  Simplex iterations : 0
  Barrier iterations : 0
  Node count         : 0


Root node processing (before b&c):
  Real time             =    0.04 sec. (53.06 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.

A JuMP Model
Minimization problem with:
Variables: 1737
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 278 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 1795 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 84 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 381 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 1356 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: CPLEX
Names registered in the model: inv, prod, q, w, x, y, z

In [204]:
function Resolve_PRP_bis(p)
    # LP = Model(HiGHS.Optimizer)
    LP = Model(CPLEX.Optimizer)
    
    c = calcul_dist(p)
    
    n = p["Clients"].nb_clients      # Nombre de clients (usine inclue)
    l = p["l"]                       # Nombre de périodes
    
    inv_0 = []
    for i in 1:n
        append!(inv_0, [p["Clients"].L0[i]])
    end
    
    @variable(LP, y[1:l], Bin)  # Production démarrée
    @variable(LP, prod[1:l], Int)  # Quantité produite
    @variable(LP, inv[1:n, 1:l], Int)  # Inventaire des clients - Défini sur T {1, ..., T}
    @variable(LP, x[1:n, 1:n, 1:l], Bin)  # Camion entre i et j au temps t
    @variable(LP, q[1:n, 1:l], Int) # qt délivré au client i à la période t
    @variable(LP, z[1:n, 1:l], Int) # 1 client i visité au temps t, 0 sinon (Attention z[0, t] nb de véhicules 
                                    # qui quittent l'usine au temps t)
    @variable(LP, w[1:n, 1:l], Int) # chargement véhicule avant de faire une livraison au client i au temps t 

    @objective(LP, Min, sum(p["u"]*prod[t] + p["f"]*y[t] + 
            sum(p["Clients"].h[i]*inv[i, t] for i in 1:n) + 
            sum(sum(c[i, j] * x[i, j, t] for i in 1:n if i ≠ j) for j in 1:n) for t in 1:l))
    
    indices_array = [i for i in 1:n-1]
    allS = partiesliste(indices_array, 1, n-2)
    
    for t in 1:l # Pour toutes les périodes (l=nb de prériodes)
        if(t ≠ 1) 
            @constraint(LP, inv[1, t-1] + prod[t] == sum(q[i, t] for i in 2:n) + inv[1, t]) # (2)
        else
            @constraint(LP, inv_0[1] + prod[t] == sum(q[i, t] for i in 2:n) + inv[1, t]) # (2)
        end
        
        sum_d_current_t = sum(sum(p["Clients"].d[ii][tt] for ii in 2:n) for tt in t:l) 
        Mt = min(p["C"], sum_d_current_t)
        #@constraint(LP, prod[t] <= Mt * y[t]) # (4)
        #@constraint(LP, inv[1, t] <= p["Clients"].L[1]) # (5)
        #@constraint(LP, z[1, t] <= p["k"]) # (10)
        
        for i in 1:n
            # (13.1) -> prod binaire donc pas besoin
            @constraint(LP, 0 <= inv[i, t]) # (13.2)
            #@constraint(LP, 0 <= q[i, t]) # (13.3)
            #@constraint(LP, 0 <= x[i, j, t] <= 1) # (14.2)
            
            sum_d_current_i = sum(p["Clients"].d[i][tt] for tt in 1:l)
            Mit = min(p["Clients"].L[i], p["Q"], sum_d_current_i)
            xijt_sum = sum(x[i, jj, t] for jj in 1:n)
            
            #@constraint(LP, xijt_sum + sum(x[jj, i, t] for jj in 1:n) == 2 * z[i, t]) # (9)
            
            if(i ≠ 1)
                if(t ≠ 1)
                    @constraint(LP, inv[i, t-1] + q[i, t] == p["Clients"].d[i][t] + inv[i, t]) # (3)
                    #@constraint(LP, inv[i, t-1] + q[i, t] <= p["Clients"].L[i]) # (6)
                else
                    @constraint(LP, inv_0[1] + q[i, t] == p["Clients"].d[i][t] + inv[i, t+1]) # (3)
                    #@constraint(LP, inv_0[1] + q[i, t] <= p["Clients"].L[i]) # (6)
                end
                #@constraint(LP, q[i, t] <= Mit * z[i, t]) # (7)
                #@constraint(LP, xijt_sum == z[i, t]) # (8)
                #@constraint(LP, 0 <= w[i, t]) # (12.1)
                #@constraint(LP, w[i, t] <= p["Q"] * z[i, t]) # (12.2)
                
                #@constraint(LP, 0 <= z[i, t] <= 1) # (15)
            end
            for j in 1:n
                if(i ≠ j)
                    #@constraint(LP, w[i, t] - w[j, t] <= q[i, t] - Mit * (1 - x[i, j, t])) # (11) OK
                end
            end
        end
    end
        
    
    # print(LP)
    # println()
    optimize!(LP)
   
    println(solution_summary(LP, verbose=true))
    
    return LP
end

Resolve_PRP_bis (generic function with 1 method)

In [205]:
LP = Resolve_PRP_bis(Read_PRP_instance("./PRP_instances/A_014_#ABS1_15_1.prp"))

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Dual infeasible due to empty column 'x8'.
Presolve time = 0.00 sec. (0.44 ticks)
* Solver : CPLEX

* Status
  Termination status : INFEASIBLE_OR_UNBOUNDED
  Primal status      : NO_SOLUTION
  Dual status        : NO_SOLUTION
  Result count       : 0
  Has duals          : false
  Message from the solver:
  "integer infeasible or unbounded"

* Candidate solution
  Objective bound      : -1.00000e+75

* Work counters
  Solve time (sec)   : 1.84298e-03
  Simplex iterations : 0
  Barrier iterations : 0
  Node count         : 0


Root node processing (before b&c):
  Real time             =    0.00 sec. (0.53 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.53 ticks)
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLE

A JuMP Model
Minimization problem with:
Variables: 1722
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 90 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 90 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 366 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 1356 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: CPLEX
Names registered in the model: inv, prod, q, w, x, y, z

In [242]:
function Resolve_PRP_ter(p)
    # LP = Model(HiGHS.Optimizer)
    LP = Model(CPLEX.Optimizer)
    
    c = calcul_dist(p)
    
    n = p["Clients"].nb_clients      # Nombre de clients (usine inclue)
    l = p["l"]                       # Nombre de périodes
    
    @variable(LP, y[1:l+1], Bin)  # Production démarrée
    @variable(LP, prod[1:l+1], Int)  # Quantité produite
    @variable(LP, inv[1:n, 1:l+1], Int)  # Inventaire des clients - Défini sur T {1, ..., T}
    @variable(LP, x[1:n, 1:n, 1:l+1], Bin)  # Camion entre i et j au temps t
    @variable(LP, q[1:n, 1:l+1], Int) # qt délivré au client i à la période t
    @variable(LP, z[1:n, 1:l+1], Int) # 1 client i visité au temps t, 0 sinon (Attention z[0, t] nb de véhicules 
                                    # qui quittent l'usine au temps t)
    @variable(LP, w[1:n, 1:l+1], Int) # chargement véhicule avant de faire une livraison au client i au temps t 

    @objective(LP, Min, sum(p["u"]*prod[t] + p["f"]*y[t] + 
            sum(p["Clients"].h[i]*inv[i, t] for i in 1:n) + 
            sum(sum(c[i, j] * x[i, j, t] for i in 1:n if i ≠ j) for j in 1:n) for t in 2:l+1))
    
    
    for i in 1:n
        @constraint(LP, inv[i, 1] == p["Clients"].L0[i]) # (2)
    end
    
    for t in 2:l+1 # Pour toutes les périodes (l=nb de prériodes)
        @constraint(LP, inv[1, t-1] + prod[t] == sum(q[i, t] for i in 2:n) + inv[1, t]) # (2)
        sum_d_current_t = sum(sum(p["Clients"].d[ii][tt] for ii in 2:n) for tt in t:l+1) 
        Mt = min(p["C"], sum_d_current_t)
        @constraint(LP, prod[t] <= Mt * y[t]) # (4)
        @constraint(LP, inv[1, t] <= p["Clients"].L[1]) # (5)
        @constraint(LP, z[1, t] <= p["k"]) # (10)
        for i in 1:n
            # (13.1) -> prod binaire donc pas besoin
            @constraint(LP, 0 <= inv[i, t]) # (13.2)
            @constraint(LP, 0 <= q[i, t]) # (13.3)
            
            sum_d_current_i = sum(p["Clients"].d[i][tt] for tt in 1:l)
            Mit = min(p["Clients"].L[i], p["Q"], sum_d_current_i)
            xijt_sum = sum(x[i, jj, t] for jj in 1:n)
            
            @constraint(LP, xijt_sum + sum(x[jj, i, t] for jj in 1:n) == 2 * z[i, t]) # (9)
            
            if(i ≠ 1)
                @constraint(LP, inv[i, t-1] + q[i, t] == p["Clients"].d[i][t] + inv[i, t]) # (3)
                @constraint(LP, inv[i, t-1] + q[i, t] <= p["Clients"].L[i]) # (6)
                @constraint(LP, q[i, t] <= Mit * z[i, t]) # (7)
                @constraint(LP, xijt_sum == z[i, t]) # (8)
                @constraint(LP, 0 <= w[i, t]) # (12.1)
                @constraint(LP, w[i, t] <= p["Q"] * z[i, t]) # (12.2)
                
                @constraint(LP, 0 <= z[i, t] <= 1) # (15)
            end
            for j in 1:n
                if(i ≠ j)
                    #@constraint(LP, w[i, t] - w[j, t] <= q[i, t] - Mit * (1 - x[i, j, t])) # (11) OK
                end
            end
        end
    end
        
    
    # print(LP)
    # println()
    optimize!(LP)
   
    println(solution_summary(LP, verbose=true))
    
    return LP
end

Resolve_PRP_ter (generic function with 1 method)

In [243]:
LP = Resolve_PRP_ter(Read_PRP_instance("./PRP_instances/A_014_#ABS1_15_1.prp"))

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Tried aggregator 2 times.
MIP Presolve eliminated 543 rows and 471 columns.
MIP Presolve modified 1384 coefficients.
Aggregator did 35 substitutions.
Reduced MIP has 319 rows, 1587 columns, and 3325 nonzeros.
Reduced MIP has 1440 binaries, 147 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (4.59 ticks)
Found incumbent of value 108278.429333 after 0.01 sec. (9.50 ticks)
Probing time = 0.00 sec. (2.02 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 319 rows, 1587 columns, and 3325 nonzeros.
Reduced MIP has 1440 binaries, 147 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (2.86 ticks)
Probing time = 0.00 sec. (2.01 ticks)
Clique table members: 187.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (1.64 ticks)

        Nodes                               

    w[5,2] : -0.00000e+00
    w[5,3] : -0.00000e+00
    w[5,4] : -0.00000e+00
    w[5,5] : -0.00000e+00
    w[5,6] : -0.00000e+00
    w[5,7] : -0.00000e+00
    w[6,1] : 0.00000e+00
    w[6,2] : -0.00000e+00
    w[6,3] : -0.00000e+00
    w[6,4] : -0.00000e+00
    w[6,5] : -0.00000e+00
    w[6,6] : -0.00000e+00
    w[6,7] : -0.00000e+00
    w[7,1] : 0.00000e+00
    w[7,2] : -0.00000e+00
    w[7,3] : -0.00000e+00
    w[7,4] : -0.00000e+00
    w[7,5] : -0.00000e+00
    w[7,6] : -0.00000e+00
    w[7,7] : -0.00000e+00
    w[8,1] : 0.00000e+00
    w[8,2] : -0.00000e+00
    w[8,3] : -0.00000e+00
    w[8,4] : -0.00000e+00
    w[8,5] : -0.00000e+00
    w[8,6] : -0.00000e+00
    w[8,7] : -0.00000e+00
    w[9,1] : 0.00000e+00
    w[9,2] : -0.00000e+00
    w[9,3] : -0.00000e+00
    w[9,4] : -0.00000e+00
    w[9,5] : -0.00000e+00
    w[9,6] : -0.00000e+00
    w[9,7] : -0.00000e+00
    x[1,1,1] : 0.00000e+00
    x[1,1,2] : 1.00000e+00
    x[1,1,3] : 1.00000e+00
    x[1,1,4] : 1.00000e+00
    x[1,1,5]

    x[12,11,5] : 0.00000e+00
    x[12,11,6] : 0.00000e+00
    x[12,11,7] : 0.00000e+00
    x[12,12,1] : 0.00000e+00
    x[12,12,2] : 0.00000e+00
    x[12,12,3] : 0.00000e+00
    x[12,12,4] : 0.00000e+00
    x[12,12,5] : 0.00000e+00
    x[12,12,6] : 1.00000e+00
    x[12,12,7] : 1.00000e+00
    x[12,13,1] : 0.00000e+00
    x[12,13,2] : 0.00000e+00
    x[12,13,3] : 0.00000e+00
    x[12,13,4] : 0.00000e+00
    x[12,13,5] : 0.00000e+00
    x[12,13,6] : 0.00000e+00
    x[12,13,7] : 0.00000e+00
    x[12,14,1] : 0.00000e+00
    x[12,14,2] : 0.00000e+00
    x[12,14,3] : 0.00000e+00
    x[12,14,4] : 0.00000e+00
    x[12,14,5] : 0.00000e+00
    x[12,14,6] : 0.00000e+00
    x[12,14,7] : 0.00000e+00
    x[12,15,1] : 0.00000e+00
    x[12,15,2] : 0.00000e+00
    x[12,15,3] : 0.00000e+00
    x[12,15,4] : 0.00000e+00
    x[12,15,5] : 0.00000e+00
    x[12,15,6] : 0.00000e+00
    x[12,15,7] : 0.00000e+00
    x[12,2,1] : 0.00000e+00
    x[12,2,2] : 0.00000e+00
    x[12,2,3] : 0.00000e+00
    x[12,2,4] : 0

    x[15,4,2] : 0.00000e+00
    x[15,4,3] : 0.00000e+00
    x[15,4,4] : 0.00000e+00
    x[15,4,5] : 0.00000e+00
    x[15,4,6] : 0.00000e+00
    x[15,4,7] : 0.00000e+00
    x[15,5,1] : 0.00000e+00
    x[15,5,2] : 0.00000e+00
    x[15,5,3] : 0.00000e+00
    x[15,5,4] : 0.00000e+00
    x[15,5,5] : 0.00000e+00
    x[15,5,6] : 0.00000e+00
    x[15,5,7] : 0.00000e+00
    x[15,6,1] : 0.00000e+00
    x[15,6,2] : 0.00000e+00
    x[15,6,3] : 0.00000e+00
    x[15,6,4] : 0.00000e+00
    x[15,6,5] : 0.00000e+00
    x[15,6,6] : 0.00000e+00
    x[15,6,7] : 0.00000e+00
    x[15,7,1] : 0.00000e+00
    x[15,7,2] : 0.00000e+00
    x[15,7,3] : 0.00000e+00
    x[15,7,4] : 0.00000e+00
    x[15,7,5] : 0.00000e+00
    x[15,7,6] : 0.00000e+00
    x[15,7,7] : 0.00000e+00
    x[15,8,1] : 0.00000e+00
    x[15,8,2] : 0.00000e+00
    x[15,8,3] : 0.00000e+00
    x[15,8,4] : 0.00000e+00
    x[15,8,5] : 0.00000e+00
    x[15,8,6] : 0.00000e+00
    x[15,8,7] : 0.00000e+00
    x[15,9,1] : 0.00000e+00
    x[15,9,2] : 0.00

    x[5,11,3] : 0.00000e+00
    x[5,11,4] : 0.00000e+00
    x[5,11,5] : 0.00000e+00
    x[5,11,6] : 0.00000e+00
    x[5,11,7] : 0.00000e+00
    x[5,12,1] : 0.00000e+00
    x[5,12,2] : 0.00000e+00
    x[5,12,3] : 0.00000e+00
    x[5,12,4] : 0.00000e+00
    x[5,12,5] : 0.00000e+00
    x[5,12,6] : 0.00000e+00
    x[5,12,7] : 0.00000e+00
    x[5,13,1] : 0.00000e+00
    x[5,13,2] : 0.00000e+00
    x[5,13,3] : 0.00000e+00
    x[5,13,4] : 0.00000e+00
    x[5,13,5] : 0.00000e+00
    x[5,13,6] : 0.00000e+00
    x[5,13,7] : 0.00000e+00
    x[5,14,1] : 0.00000e+00
    x[5,14,2] : 0.00000e+00
    x[5,14,3] : 0.00000e+00
    x[5,14,4] : 0.00000e+00
    x[5,14,5] : 0.00000e+00
    x[5,14,6] : 0.00000e+00
    x[5,14,7] : 0.00000e+00
    x[5,15,1] : 0.00000e+00
    x[5,15,2] : 0.00000e+00
    x[5,15,3] : 0.00000e+00
    x[5,15,4] : 0.00000e+00
    x[5,15,5] : 0.00000e+00
    x[5,15,6] : 0.00000e+00
    x[5,15,7] : 0.00000e+00
    x[5,2,1] : 0.00000e+00
    x[5,2,2] : 0.00000e+00
    x[5,2,3] : 0.00000

    x[8,5,6] : 0.00000e+00
    x[8,5,7] : 0.00000e+00
    x[8,6,1] : 0.00000e+00
    x[8,6,2] : 0.00000e+00
    x[8,6,3] : 0.00000e+00
    x[8,6,4] : 0.00000e+00
    x[8,6,5] : 0.00000e+00
    x[8,6,6] : 0.00000e+00
    x[8,6,7] : 0.00000e+00
    x[8,7,1] : 0.00000e+00
    x[8,7,2] : 0.00000e+00
    x[8,7,3] : 0.00000e+00
    x[8,7,4] : 0.00000e+00
    x[8,7,5] : 0.00000e+00
    x[8,7,6] : 0.00000e+00
    x[8,7,7] : 0.00000e+00
    x[8,8,1] : 0.00000e+00
    x[8,8,2] : 0.00000e+00
    x[8,8,3] : 0.00000e+00
    x[8,8,4] : 0.00000e+00
    x[8,8,5] : 0.00000e+00
    x[8,8,6] : 0.00000e+00
    x[8,8,7] : 1.00000e+00
    x[8,9,1] : 0.00000e+00
    x[8,9,2] : 0.00000e+00
    x[8,9,3] : 0.00000e+00
    x[8,9,4] : 0.00000e+00
    x[8,9,5] : 0.00000e+00
    x[8,9,6] : 0.00000e+00
    x[8,9,7] : 0.00000e+00
    x[9,1,1] : 0.00000e+00
    x[9,1,2] : 0.00000e+00
    x[9,1,3] : 0.00000e+00
    x[9,1,4] : 0.00000e+00
    x[9,1,5] : 0.00000e+00
    x[9,1,6] : 0.00000e+00
    x[9,1,7] : 0.00000e+00
 

A JuMP Model
Minimization problem with:
Variables: 2009
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 279 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 534 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 84 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 427 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 1582 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: CPLEX
Names registered in the model: inv, prod, q, w, x, y, z

In [ ]:
function Resolve_PRP2(p)
    LP = Model(CPLEX.Optimizer)
    
    c = calcul_dist(p)
    
    n = p["Clients"].nb_clients      # Nombre de clients (usine inclue)
    l = p["l"]                       # Nombre de périodes
    
    @variable(LP, prod[1:l+1], Int)
    @variable(LP, inv[1:n, 1:l+1], Int)
    @variable(LP, y[1:l+1], Bin)
    @variable(LP, z[1:n, 1:l+1], Int)
    @variable(LP, x[1:n, 1:n, 1:l+1], Int)
    @variable(LP, q[1:n, 1:l+1], Bin)
    @variable(LP, w[1:n, 1:l+1], Int)
    
    @objective(LP, Min, sum(p["u"]*prod[t] + p["f"]*y[t] +
            sum(p["Clients"].h[i]*inv[i, t] for i in 1:n) +
            sum(sum(c[i, j] for j in 1:n if i ≠ j) for i in 1:n) * sum() for t in 2:l+1))
    
    
end